# gu_toolkit: Efficient Guide, Compendium, and Applications

This notebook is an opinionated, practical guide for everyday toolkit use. It is organized for **fast onboarding** and **copy/paste utility**.

## How to use this notebook
1. Run the setup cell.
2. Jump to the section you need (quickstart, recipes, applications).
3. Copy a pattern and adapt it.

---

## Contents
- Quickstart workflow
- Frequently used APIs (compendium)
- Parameter and numeric-expression patterns
- Introspection and debugging helpers
- Cool applications (interactive demos)


In [ ]:
# Setup
import sys
from pathlib import Path

try:
    _start = Path(__file__).resolve().parent
except NameError:
    _start = Path.cwd().resolve()

_pkg_root = _start
while _pkg_root != _pkg_root.parent and not (_pkg_root / "__init__.py").exists():
    _pkg_root = _pkg_root.parent
sys.path.insert(0, str(_pkg_root.parent))

import math
import numpy as np

from gu_toolkit import *


## 1) Quickstart: Minimal interactive figure

This pattern covers most exploratory notebook work:
- create a figure,
- define one or more plots,
- bind parameters by name,
- render.


In [ ]:
fig = Figure(x_range=(-8, 8), y_range=(-3, 3), sampling_points=500)

plot(lambda x, a, b: a*np.sin(b*x), id="wave", label="a*sin(bx)")
parameter("a", value=1.0, min=-2.0, max=2.0, step=0.1)
parameter("b", value=1.0, min=0.2, max=4.0, step=0.1)

set_title("Quickstart: Interactive sine")
render()

## 2) Compendium: Frequently used operations

### Figure construction and ranges
- `Figure(...)`
- `get_x_range()`, `get_y_range()`
- `set_title(...)`

### Plot and parameter workflow
- `plot(...)`
- `parameter(name, ...)`
- `parameters(...)` / `params(...)`
- `render()`

### Numeric expressions
- `numpify(expr, vars=...)`
- `DYNAMIC_PARAMETER` and `UNFREEZE`

### Symbolic parsing
- `parse_latex(latex_string)`


In [ ]:
# Compact utility snippets
print("x-range:", get_x_range())
print("y-range:", get_y_range())
print("param snapshot:", params())

## 3) Parameter patterns you will reuse often

### A) Explicit slider bounds and defaults
Useful for reproducible demos.


In [ ]:
fig2 = Figure(x_range=(-10, 10), y_range=(-2, 2), sampling_points=600)
plot(lambda x, amp, freq, phase: amp*np.cos(freq*x + phase), id="cosine")

parameters(
    amp={"value": 1.0, "min": 0.0, "max": 2.0, "step": 0.05},
    freq={"value": 1.0, "min": 0.1, "max": 3.0, "step": 0.05},
    phase={"value": 0.0, "min": -math.pi, "max": math.pi, "step": 0.1},
)
set_title("Cosine with amplitude/frequency/phase controls")
render()

### B) Symbolic-to-numeric with dynamic parameters

Use this when you want to start from symbolic math and still keep interactivity.


In [ ]:
expr = parse_latex(r"A \sin(k x + \phi)")
f = numpify(expr, vars=["x", DYNAMIC_PARAMETER, DYNAMIC_PARAMETER, DYNAMIC_PARAMETER])

fig3 = Figure(x_range=(-8, 8), y_range=(-3, 3), sampling_points=700)
plot(lambda x, A, k, phi: f(x, A, k, phi), id="symbolic-wave")
parameters(
    A={"value": 1.0, "min": 0.0, "max": 2.0, "step": 0.1},
    k={"value": 1.0, "min": 0.2, "max": 4.0, "step": 0.1},
    phi={"value": 0.0, "min": -math.pi, "max": math.pi, "step": 0.1},
)
set_title("Symbolic expression with dynamic parameters")
render()

## 4) Debugging and introspection helpers

These checks are useful when a notebook cell is not behaving as expected.


In [ ]:
print("Current params:", params())
print("Figure has", len(fig3.plots), "plot(s)")
print("Plot ids:", list(fig3.plots.keys()))

plot_obj = fig3.plots["symbolic-wave"]
print("Plot label:", plot_obj.label)
print("Has cached samples:", plot_obj.cached_samples is not None)

## 5) Cool applications

### Application 1: Harmonic synthesizer
A compact additive synthesis visualization.


In [ ]:
fig4 = Figure(x_range=(-2*np.pi, 2*np.pi), y_range=(-4, 4), sampling_points=900)

plot(lambda x, a1, a2, a3: a1*np.sin(x) + a2*np.sin(2*x) + a3*np.sin(3*x), id="harmonic")
parameters(
    a1={"value": 1.0, "min": -2.0, "max": 2.0, "step": 0.1},
    a2={"value": 0.0, "min": -2.0, "max": 2.0, "step": 0.1},
    a3={"value": 0.0, "min": -2.0, "max": 2.0, "step": 0.1},
)
set_title("Harmonic synthesizer")
render()

### Application 2: Gaussian mixture explorer
Great for intuition about peak width, center, and mixture weights.


In [ ]:
def gauss(x, mu, sigma):
    return np.exp(-0.5*((x-mu)/sigma)**2)

fig5 = Figure(x_range=(-8, 8), y_range=(0, 2), sampling_points=800)
plot(lambda x, w1, w2, m1, m2, s1, s2: w1*gauss(x, m1, s1) + w2*gauss(x, m2, s2), id="mixture")
parameters(
    w1={"value": 1.0, "min": 0.0, "max": 1.5, "step": 0.05},
    w2={"value": 0.8, "min": 0.0, "max": 1.5, "step": 0.05},
    m1={"value": -1.5, "min": -5.0, "max": 5.0, "step": 0.1},
    m2={"value": 1.5, "min": -5.0, "max": 5.0, "step": 0.1},
    s1={"value": 1.0, "min": 0.2, "max": 3.0, "step": 0.1},
    s2={"value": 0.8, "min": 0.2, "max": 3.0, "step": 0.1},
)
set_title("Gaussian mixture explorer")
render()

## 6) Practical checklist

When prototyping in notebooks:
- Start with one plot and one parameter.
- Confirm `params()` updates while sliders move.
- Increase `sampling_points` only when needed.
- Use clear `id=` values for plots you update repeatedly.

This notebook is intended as a durable daily-use reference.
